In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
train_data = pd.read_csv('./data/train.csv')
test_data= pd.read_csv('./data/test.csv')

In [ ]:
numeric_features_train = train_data.select_dtypes(include=['float64', 'int64']).columns.tolist()

Como os valores ausentes são puramente categóricos, não a necessidade explícita de tratá-los (os nomes do artista, album e música não importam) para o modelo preditivo. Então segue com a remoção dos outliers.

In [ ]:
# Função para remover outliers utilizando o método IQR
def remove_outliers_iqr(df, numeric_features):
    for col in numeric_features:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Definindo limites inferior e superior
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Remover outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

In [ ]:
train_data_cleaned = remove_outliers_iqr(train_data, numeric_features_train)

# Verificando o tamanho do novo DataFrame
print(f"Tamanho original do conjunto de teste: {train_data.shape[0]}")
print(f"Tamanho após remoção de outliers: {train_data.shape[0]}")

Foi notado que não há outliers na tabela

## Hipótese 1: 
Há alguns gêneros que possuem significativamente mais músicas populares do que outros, portanto há uma relação direta. Posso atribuir pesos aos gêneros e isso afetará positivamente o resultado da IA.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Contar a frequência de músicas populares e não populares por gênero
genre_popularity = train_data.groupby(['track_genre', 'popularity_target']).size().unstack()

# 2. Aumentar o tamanho da figura para comportar mais gêneros
plt.figure(figsize=(20, 8))  # Aumenta a largura da figura

# 3. Plotar o gráfico de barras empilhadas com mais espaço
genre_popularity.plot(kind='bar', stacked=True, color=['skyblue', 'orange'], figsize=(20, 8))

# 4. Ajustes no gráfico
plt.title('Distribuição de Popularidade por Gênero Musical', fontsize=16)
plt.xlabel('Gênero Musical', fontsize=12)
plt.ylabel('Número de Músicas', fontsize=12)

# Rotaciona as labels do eixo X para evitar sobreposição
plt.xticks(rotation=90, ha='center')  

plt.legend(title='Popularidade', labels=['Não Popular', 'Popular'])
plt.tight_layout()

# 5. Mostrar o gráfico
plt.show()


## Hipótese 2:
Há uma relação direta entre danceability e loudness (quanto mais 'dançável' a música, mais alta ela é), dado algumas exceções. Sabendo disso, posso utilizar PCA na tabela para melhorar os resultados no modelo.

In [ ]:
# Amostrar os dados para evitar poluição
train_sample = train_data.sample(1000, random_state=42)  # Reduzir o número de pontos exibidos

# Criar o gráfico de dispersão com transparência ajustada
plt.figure(figsize=(8, 6))
sns.set(style="whitegrid")

scatter = sns.scatterplot(data=train_sample, x='danceability', y='loudness', 
                          hue='popularity_target', palette='coolwarm', 
                          alpha=0.6, s=50, edgecolor=None)

# Adicionar rótulos e título
plt.title('Relação entre Danceability e Loudness', fontsize=16, pad=20)
plt.xlabel('Danceability', fontsize=12, labelpad=10)
plt.ylabel('Loudness', fontsize=12, labelpad=10)

# Remover a legenda para simplicidade
scatter.legend_.remove()

# Exibir o gráfico
plt.show()


## Hipótese 3:
Quanto mais energia a música tem, maior a sua dançabilidade no geral. Com isso, podemos ver uma relação direta entre essas features.

In [ ]:
# Ajustar a amostra de dados para evitar poluição (opcional)
train_sample = train_data.sample(1000, random_state=42)  # Reduzir o número de pontos exibidos

# Definir o tamanho do gráfico e estilo
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")

# Criar o gráfico de dispersão com transparência ajustada
scatter = sns.scatterplot(x='energy', y='danceability', data=train_sample, hue='popularity_target', palette='coolwarm', alpha=0.5, s=70, edgecolor=None)

# Adicionar rótulos e título
plt.title('Relação entre Energy e Danceability (Amostra)', fontsize=16, pad=20)
plt.xlabel('Energy', fontsize=12, labelpad=10)
plt.ylabel('Danceability', fontsize=12, labelpad=10)

# Remover a legenda para simplicidade
scatter.legend_.remove()

# Exibir o gráfico
plt.show()


In [ ]:
# 1. Calcular a proporção de músicas populares para cada gênero
genre_popularity = train_data.groupby('track_genre')['popularity_target'].mean()

# 2. Normalizar os pesos para que somem 1 (ou para que tenham uma escala comparável)
weights = genre_popularity / genre_popularity.sum()

In [ ]:
# 1. Adicionar os pesos calculados como uma nova coluna no conjunto de treino
train_data['genre_weight'] = train_data['track_genre'].map(weights)

In [ ]:
# 2. Separar as features e a target
X_train = train_data.drop(columns=['popularity_target', 'genre_weight', 'track_id', 'track_unique_id', 'key', 'album_name', 'track_name', 'explicit', 'duration_ms', 'time_signature', 'mode'])
y_train = train_data['popularity_target']

X_test = test_data  

In [ ]:
# 3. Definir colunas numéricas e categóricas
numeric_features = ['danceability', 'energy', 'loudness', 'valence', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']
categorical_features = ['track_genre']

In [ ]:
# 4. Criar o pré-processador para tratar as variáveis numéricas e categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [ ]:
# 5. Criar pipeline com pré-processamento e modelo de regressão logística
logistic_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

In [ ]:
# 6. Dividir o conjunto de treino em treino (80%) e validação (20%)
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# 7. Treinar o modelo no conjunto de treino dividido
logistic_model.fit(X_train_split, y_train_split)

In [ ]:
# 8. Fazer previsões no conjunto de validação
y_val_pred = logistic_model.predict(X_val)

In [ ]:
# 9. Calcular e exibir as métricas de avaliação
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Acurácia: {accuracy:.2f}')

In [ ]:
# plot da matriz de confusão
conf_matrix = confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Popular', 'Popular'], yticklabels=['Não Popular', 'Popular'])
plt.xlabel('Predição')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Relatório de classificação detalhado
print('Relatório de Classificação:')
print(classification_report(y_val, y_val_pred))

Agora vamos testar com a hipótese 1: relacionando os pesos

In [ ]:
logistic_model.fit(X_train, y_train, classifier__sample_weight=train_data['genre_weight'])
y_val_pred = logistic_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Acurácia: {accuracy:.2f}')


In [ ]:
# plot da matriz de confusão
conf_matrix = confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Popular', 'Popular'], yticklabels=['Não Popular', 'Popular'])
plt.xlabel('Predição')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

Agora, vamos testar random forest para o treinamento do modelo

In [ ]:
# 5. Criar pipeline com pré-processamento e modelo de regressão logística
random_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))
])

In [ ]:
# Treinar o modelo no conjunto de treino dividido com RandomForest
random_model.fit(X_train_split, y_train_split)
y_val_pred = random_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Acurácia: {accuracy:.2f}')


In [ ]:
# plot da matriz de confusão
conf_matrix = confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Popular', 'Popular'], yticklabels=['Não Popular', 'Popular'])
plt.xlabel('Predição')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

Agora vamos aplicar FineTunning para os hiperparâmetros de ambos os modelos

In [ ]:
# 1. Definindo os hiperparâmetros para Logistic Regression
logistic_param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10],
    'classifier__penalty': ['l1', 'l2'],  # Use 'elasticnet' se max_iter >= 1000
}

In [ ]:
# Criar o objeto GridSearchCV para Logistic Regression
logistic_grid_search = GridSearchCV(logistic_model, logistic_param_grid, cv=5, scoring='accuracy')

In [ ]:
# Ajustar o GridSearchCV aos dados de treinamento
logistic_grid_search.fit(X_train_split, y_train_split)

In [ ]:
# Exibir os melhores parâmetros
print("Melhores parâmetros para Logistic Regression:")
print(logistic_grid_search.best_params_)

In [ ]:
# 5. Criar pipeline com pré-processamento e modelo de regressão logística
logistic_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000, C=10, penalty='l2'))
])

In [ ]:
logistic_model.fit(X_train, y_train)
y_val_pred = logistic_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Acurácia: {accuracy:.2f}')

In [ ]:
random_param_grid = {
    'classifier__n_estimators': [50, 100, 200],  
    'classifier__max_depth': [None, 10, 20],  
}

In [ ]:
rf_grid_search = GridSearchCV(random_model, random_param_grid, cv=5, scoring='accuracy')

In [ ]:
# Ajustar o GridSearchCV aos dados de treinamento
rf_grid_search.fit(X_train_split, y_train_split)

In [ ]:
# Exibir os melhores parâmetros
print("Melhores parâmetros para Random Forest Classifier:")
print(rf_grid_search.best_params_)

In [ ]:
random_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=200, max_depth=None))
])

In [ ]:
# Treinar o modelo no conjunto de treino dividido com RandomForest
random_model.fit(X_train_split, y_train_split)
y_val_pred = random_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Acurácia: {accuracy:.2f}')  

In [ ]:
# 10. Fazer previsões no conjunto de teste
y_test_pred = random_model.predict(X_test)

# Caso tenha o arquivo de submissão
submission = pd.DataFrame({'track_unique_id': test_data['track_unique_id'], 'popularity_target': y_test_pred})

# 12. Salvar o arquivo de submissão
submission.to_csv('./submission.csv', index=False)

print('Arquivo de submissão gerado com sucesso!')